In [1]:
import sys
import pandas as pd
import nibabel as nib
import nibabel.processing as prc
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as mt
import math

In [2]:
subjectDir = str('/media/corey/2_4TB-WDBlue/DTI_merged_data/')
DirExtName = str('/4o/averageFAs/Omuircheartaigh/')
FA_filenames = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/DTI/Omuir_FA_list.txt',dtype=str)
#Load subjectlist of interest, store IDs as integers
sub_list = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/subsWithDTI.txt',dtype=str)
#Store subject names as a list with a standard suffix if provided
sub_names = sub_list

subjectInfo = pd.read_csv(open('/media/corey/4TB-WDBlue/data-thesis/subject_info.csv'),delim_whitespace=True,usecols=(4,11))

In [3]:
control_FA = []
nonaff_FA = []
aff_FA = []
n_control = 0
n_aff = 0
n_nonaff = 0
for n in sub_names:
    path_to_data = subjectDir + n + DirExtName
    phen = subjectInfo.loc[subjectInfo['src_subject_id'] == n]
    pdesc = phen['phenotype_description'].values[0]
    for r in FA_filenames:
        fa_filename = path_to_data + r
        fa_file = nib.load(fa_filename)
        fa_data = fa_file.get_fdata()
        FA_val = np.max(fa_data)
        if pdesc == 'In,good,health':
            control_FA.append([FA_val, r])
        if pdesc == 'Non-affective,psychosis':
            nonaff_FA.append([FA_val, r])
        if pdesc == 'Affective,psychosis':
            aff_FA.append([FA_val, r])
    if pdesc == 'In,good,health':
        n_control += 1
    if pdesc == 'Non-affective,psychosis':
        n_nonaff +=1
    if pdesc == 'Affective,psychosis':
        n_aff += 1

In [4]:
c_regionwise_struct = []
n_rw_struct = []
a_rw_struct = []

all_FAs_c = []
mean_FAs_c = []
stdv_FA_c = []

all_FAs_n = []
mean_FAs_n = []
stdv_FA_n = []

all_FAs_a = []
mean_FAs_a = []
stdv_FA_a = []

curr_region = 0
for r in FA_filenames:
    regional_c = [tuple for tuple in control_FA if any(r == i for i in tuple)] #Sort data by region
    reg_n = [tuple for tuple in nonaff_FA if any(r == i for i in tuple)]
    reg_a = [tuple for tuple in aff_FA if any(r == i for i in tuple)]

    c_regionwise_struct.append(regional_c) #Create list of tuples for each group
    n_rw_struct.append(reg_n)
    a_rw_struct.append(reg_a)

    fullc = [list(zip(*c_regionwise_struct[curr_region]))[0],r]
    meanc = [np.mean(list(zip(*c_regionwise_struct[curr_region]))[0]),r]
    stdvc = [np.std(list(zip(*c_regionwise_struct[curr_region]))[0]),r]
    fulln = [list(zip(*n_rw_struct[curr_region]))[0],r]
    meann = [np.mean(list(zip(*n_rw_struct[curr_region]))[0]),r]
    stdvn = [np.std(list(zip(*n_rw_struct[curr_region]))[0]),r]
    fulla = [list(zip(*a_rw_struct[curr_region]))[0],r]
    meana = [np.mean(list(zip(*a_rw_struct[curr_region]))[0]),r]
    stdva = [np.std(list(zip(*a_rw_struct[curr_region]))[0]),r]

    all_FAs_c.append(fullc) #Store all, mean, and standard deviation values in list including region labels
    mean_FAs_c.append(meanc)
    stdv_FA_c.append(stdvc)

    all_FAs_n.append(fulln)
    mean_FAs_n.append(meann)
    stdv_FA_n.append(stdvn)

    all_FAs_a.append(fulla)
    mean_FAs_a.append(meana)
    stdv_FA_a.append(stdva)

    curr_region += 1 #Increase index counter

In [5]:
#Calculate mean and standard deviation for the FA values in each group and perform subsequent T-tests for differences in means with heterogenous variance
Difference_C_N = []

list_abs_diff_cn = []
list_diff_cn_pvals = []
list_diff_cn_CI_L = []
list_diff_cn_CI_U = []
 
x = enumerate(FA_filenames)
for r in x:
    #Difference in Control vs Non-affective Psychosis Fractional Anisotropy
    test_c_n = stats.ttest_ind(all_FAs_c[r[0]][0],all_FAs_n[r[0]][0],equal_var=False)
    abs_diff_cn = np.mean(all_FAs_c[r[0]][0]) - np.mean(all_FAs_n[r[0]][0])
    
    ci_cn = test_c_n.confidence_interval(confidence_level=0.95)
    if test_c_n[1] <= 0.05:
        store_c_n = [abs_diff_cn, test_c_n[0], test_c_n[1], r[1], ci_cn]
        Difference_C_N.append(store_c_n)
        
    list_abs_diff_cn.append(abs_diff_cn)
    list_diff_cn_pvals.append(test_c_n[1])
    list_diff_cn_CI_L.append(test_c_n.confidence_interval(confidence_level=0.95)[0])
    list_diff_cn_CI_U.append(test_c_n.confidence_interval(confidence_level=0.95)[1])


In [6]:
fdr_corrected_pvals = mt.fdrcorrection(list_diff_cn_pvals,alpha=0.05)

In [7]:
nwomuir_differences_fdr_corrected_table = pd.DataFrame(columns=['ID','\u0394','FDR P-val', 'CI-LB', 'CI-UB', 'P-val','Tal. Labels'])
wm_list = np.arange(0,50)

In [8]:
all_talairach_top_flags = [(133.0, 173.0, 51.0),
 (485.0, 88.0, 51.0),
 (347.0, 51.0, 485.0),
 (879.0, 878.0, 784.0),
 (195.0, 193.0, 155.0),
 (51.0, 157.0, 252.0),
 (4.0, 687.0, 194.0),
 (796.0, 879.0, 878.0),
 (193.0, 194.0, 195.0),
 (24.0, 51.0, 39.0),
 (309.0, 52.0, 486.0),
 (798.0, 194.0, 880.0),
 (860.0, 864.0, 968.0),
 (798.0, 786.0, 881.0),
 (347.0, 439.0, 39.0),
 (193.0, 347.0, 195.0),
 (134.0, 174.0, 664.0),
 (194.0, 687.0, 196.0),
 (685.0, 193.0, 784.0),
 (194.0, 196.0, 28.0),
 (922.0, 972.0, 107.0),
 (194.0, 348.0, 156.0),
 (230.0, 231.0, 107.0),
 (348.0, 231.0, 156.0),
 (193.0, 194.0, 687.0),
 (193.0, 685.0, 682.0),
 (28.0, 42.0, 52.0),
 (348.0, 4.0, 89.0),
 (89.0, 486.0, 253.0),
 (42.0, 52.0, 28.0),
 (309.0, 52.0, 158.0),
 (52.0, 253.0, 28.0),
 (860.0, 51.0, 308.0),
 (196.0, 156.0, 194.0),
 (921.0, 230.0, 860.0),
 (348.0, 190.0, 194.0),
 (195.0, 193.0, 51.0),
 (798.0, 879.0, 796.0),
 (193.0, 347.0, 155.0),
 (193.0, 194.0, 347.0),
 (864.0, 52.0, 309.0),
 (51.0, 308.0, 485.0),
 (189.0, 347.0, 193.0),
 (52.0, 28.0, 194.0),
 (348.0, 42.0, 441.0),
 (922.0, 107.0, 374.0),
 (921.0, 107.0, 371.0),
 (51.0, 157.0, 24.0),
 (39.0, 24.0, 51.0),
 (39.0, 51.0, 347.0)]

In [48]:
shifted_wm_list = wm_list + 1

In [51]:
nwomuir_differences_fdr_corrected_table['ID'] = shifted_wm_list # Label IDs
nwomuir_differences_fdr_corrected_table['\u0394'] = list_abs_diff_cn #Second column is absolute difference in FA
nwomuir_differences_fdr_corrected_table['FDR P-val'] = fdr_corrected_pvals[1] #Third column is corrected pval
nwomuir_differences_fdr_corrected_table['CI-LB'] = list_diff_cn_CI_L #Fourth column is one sided confidence interval
nwomuir_differences_fdr_corrected_table['CI-UB'] = list_diff_cn_CI_U
nwomuir_differences_fdr_corrected_table['P-val'] = list_diff_cn_pvals
nwomuir_differences_fdr_corrected_table['Tal. Labels'] = all_talairach_top_flags

In [10]:
def bold_significant_ps(x):
    return 'font-weight: bold' if x <= 0.05 else ''

In [52]:
nwomuir_differences_fdr_corrected_table.style \
.format(precision=6, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.applymap(bold_significant_ps, subset=['FDR P-val', 'P-val'])

,ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
0,1,0.005390,0.219778,-0.001987,0.012767,0.150273,"(133.0, 173.0, 51.0)"
1,2,0.006236,0.149519,-0.001121,0.013593,0.095692,"(485.0, 88.0, 51.0)"
2,3,0.005463,0.088086,0.000435,0.010492,0.033473,"(347.0, 51.0, 485.0)"
3,4,0.007597,0.043444,0.001572,0.013621,0.013902,"(879.0, 878.0, 784.0)"
4,5,0.007905,0.142506,-0.001207,0.017017,0.088354,"(195.0, 193.0, 155.0)"
5,6,0.007295,0.075679,0.000837,0.013752,0.027244,"(51.0, 157.0, 252.0)"
6,7,0.001852,0.600788,-0.003790,0.007495,0.516678,"(4.0, 687.0, 194.0)"
7,8,0.001675,0.754864,-0.007211,0.010562,0.709572,"(796.0, 879.0, 878.0)"
8,9,0.002667,0.626620,-0.006181,0.011514,0.551426,"(193.0, 194.0, 195.0)"
9,10,0.005021,0.125193,-0.000319,0.010361,0.065100,"(24.0, 51.0, 39.0)"


In [53]:
stripped_nwo_table = nwomuir_differences_fdr_corrected_table.drop(nwomuir_differences_fdr_corrected_table[nwomuir_differences_fdr_corrected_table['FDR P-val'] > 0.05].index)

In [54]:
stripped_nwo_table.style \
.format(precision=6, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.applymap(bold_significant_ps, subset=['FDR P-val', 'P-val']) \
.hide()

ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
4,0.007597,0.043444,0.001572,0.013621,0.013902,"(879.0, 878.0, 784.0)"
13,0.010227,0.037536,0.002441,0.018013,0.010510,"(860.0, 864.0, 968.0)"
15,0.011908,0.007498,0.005095,0.018722,0.000750,"(347.0, 439.0, 39.0)"
16,0.011787,0.030804,0.003234,0.020340,0.007393,"(193.0, 347.0, 195.0)"
17,0.010104,0.015068,0.003761,0.016447,0.002110,"(134.0, 174.0, 664.0)"
22,0.009483,0.015068,0.003530,0.015437,0.002057,"(194.0, 348.0, 156.0)"
23,0.010559,0.025583,0.003164,0.017954,0.005628,"(230.0, 231.0, 107.0)"
24,0.009627,0.041376,0.002125,0.017129,0.012413,"(348.0, 231.0, 156.0)"
29,0.011483,0.025583,0.003449,0.019517,0.005495,"(89.0, 486.0, 253.0)"
37,0.011700,0.017844,0.004037,0.019364,0.003085,"(195.0, 193.0, 51.0)"


#Load nifti image of 50 masks (Supplementary image 9)
supp_im_9_path = ('/home/corey/atlases/Omuir2018/SupplementaryImages/SI-9-ICsTracts050_MM_2mm.nii.gz')
supp9 = nib.load(supp_im_9_path)
supp9_data = supp9.get_fdata()

Can use fslsplit to separate the mask volumes from the original 4D provided in the supplementary materials

subject_list = open('/media/corey/4TB-WDBlue/data-thesis/subject_list_full.txt')
sublist = subject_list.readlines()
ext_subjectDir = '/media/corey/2_4TB-WDBlue/ext_data/EP_FunctionalPreprocessing'
DTI_dir = '/media/corey/2_4TB-WDBlue/DTI_merged_data'

FA_test_file = nib.load('/media/corey/2_4TB-WDBlue/DTI_merged_data/1009/4o/BedpostX_1009.bedpostX/mean_fsumsamples.nii.gz')
FA_test_data = FA_test_file.get_fdata()
print(FA_test_data.shape)
FA_test_affine = FA_test_file.affine


Resample Supp9 image and Grey matter mask to 140,140,92 1.5mm^3 dimensionality

affineSupp9 = supp9.affine
for v in range(0,supp9_data.shape[3]):
#Create resampled masks [1.5mm^3] for each volume in Supplementary Image 9 (O'muircheartaigh)
    oneSliceFA = supp9_data[:,:,:,v]
    oneSliceNift = nib.Nifti1Image(oneSliceFA,affineSupp9)
    Supp9_resampled_data = prc.resample_from_to(oneSliceNift,FA_test_file,order=0)
    nib.save(Supp9_resampled_data,'/home/corey/atlases/Omuir2018/SlicedTracts/thresholded_tract_%d' %v)


import glob

#Resample grey matter masks to 1.5^3mm space (140,140,92)
for n, sub in enumerate(sublist):
    sub = sub.strip()
    GM_2mm = nib.load('%s/%s_01_MR/resampled_gm_mask.nii.gz' % (ext_subjectDir, sub))
    GM_resampledFAspace = prc.resample_from_to(GM_2mm,FA_test_file,order=0)
    nib.save(GM_resampledFAspace, '%s/%s_01_MR/resampled_gm_mask_1.5mm.nii.gz' % (ext_subjectDir, sub))

In [14]:
good_subject_list = open('/media/corey/4TB-WDBlue/data-thesis/subsWithDTI.txt')


good_sub_list = good_subject_list.readlines()  
for n, sub in enumerate(good_sub_list):
    #Load subject specific grey matter segment (SPM generated)
    sub = sub.strip()
    grey_matter_file = nib.load('%s/%s_01_MR/resampled_gm_mask_1.5mm.nii.gz' % (ext_subjectDir, sub))
    GM_data = grey_matter_file.get_fdata()
    #Load FA data (subjectwise)
    FA_file = nib.load('%s/%s/4o/BedpostX_%s.bedpostX/mean_fsumsamples.nii.gz' % (DTI_dir, sub, sub))
    FA_data = FA_file.get_fdata()
    for v in range(0,supp9_data.shape[3]):
        mask_skel = np.zeros_like(GM_data)
        curr_supp9_file = nib.load('/home/corey/atlases/Omuir2018/SlicedTracts/thresholded_tract_%d.nii' %v)
        curr_supp9_data = curr_supp9_file.get_fdata()
    #Define condition of whether voxel is in WM mask AND out of GM mask or not
        for x in range(0,curr_supp9_data.shape[0]-1):
            for y in range(0,curr_supp9_data.shape[1]-1):
                for z in range(0,curr_supp9_data.shape[2]-1):
                    if curr_supp9_data[x,y,z] != 0 and math.isnan(curr_supp9_data[x,y,z]) is False and GM_data[x,y,z] != 1:
                        #Add voxel's value to mask skeleton
                        mask_skel[x,y,z] = FA_data[x,y,z]
        num_nz_voxels = np.count_nonzero(mask_skel)
        total_signal = np.ndarray.sum(mask_skel)
        averageFA = total_signal / num_nz_voxels
        np.copyto(mask_skel, averageFA, where=mask_skel != 0)
        mask_skel_image = nib.Nifti1Image(mask_skel,FA_test_affine)
        nib.save(mask_skel_image,'%s/%s/4o/averageFAs/Omuircheartaigh/tract_%d.nii' % (DTI_dir, sub, v))
    


##########WEIGHTED TRACT_GENERATION############

good_subject_list = open('/media/corey/4TB-WDBlue/data-thesis/subsWithDTI.txt')
good_sub_list = good_subject_list.readlines()  
for n, sub in enumerate(good_sub_list):
    #Load subject specific grey matter segment (SPM generated)
    sub = sub.strip()
    grey_matter_file = nib.load('%s/%s_01_MR/resampled_gm_mask_1.5mm.nii.gz' % (ext_subjectDir, sub))
    GM_data = grey_matter_file.get_fdata()
    #Load FA data (subjectwise)
    FA_file = nib.load('%s/%s/4o/BedpostX_%s.bedpostX/mean_fsumsamples.nii.gz' % (DTI_dir, sub, sub))
    FA_data = FA_file.get_fdata()
    for v in range(0,supp9_data.shape[3]):
        mask_skel = np.zeros_like(GM_data)
        curr_supp9_file = nib.load('/home/corey/atlases/Omuir2018/SlicedTracts/thresholded_tract_%d.nii' %v)
        curr_supp9_data = curr_supp9_file.get_fdata()
    #Define condition of whether voxel is in WM mask AND out of GM mask or not
        for x in range(0,curr_supp9_data.shape[0]-1):
            for y in range(0,curr_supp9_data.shape[1]-1):
                for z in range(0,curr_supp9_data.shape[2]-1):
                    if curr_supp9_data[x,y,z] != 0 and math.isnan(curr_supp9_data[x,y,z]) is False and GM_data[x,y,z] != 1:
                        #Add voxel's value to mask skeleton
                        mask_skel[x,y,z] = FA_data[x,y,z] * curr_supp9_data[x,y,z]
        num_nz_voxels = np.count_nonzero(mask_skel)
        total_signal = np.ndarray.sum(mask_skel)
        averageFA = total_signal / num_nz_voxels
        weighted_image = nib.Nifti1Image(mask_skel,FA_test_affine)
        nib.save(weighted_image,'%s/%s/4o/averageFAs/Omuircheartaigh/tract_%d_w.nii' % (DTI_dir, sub, v))
        np.copyto(mask_skel, averageFA, where=mask_skel != 0)
        mask_skel_image = nib.Nifti1Image(mask_skel,FA_test_affine)
        nib.save(mask_skel_image,'%s/%s/4o/averageFAs/Omuircheartaigh/tract_%d_wt.nii' % (DTI_dir, sub, v))

In [15]:
wtFA_filenames = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/DTI/wt_Omuir_FA_list.txt',dtype=str)

In [16]:
wtcontrol_FA = []
wtnonaff_FA = []

n_control = 0
n_aff = 0
n_nonaff = 0
for n in sub_names:
    path_to_data = subjectDir + n + DirExtName
    phen = subjectInfo.loc[subjectInfo['src_subject_id'] == n]
    pdesc = phen['phenotype_description'].values[0]
    for r in wtFA_filenames:
        fa_filename = path_to_data + r
        fa_file = nib.load(fa_filename)
        fa_data = fa_file.get_fdata()
        FA_val = np.max(fa_data)
        if pdesc == 'In,good,health':
            wtcontrol_FA.append([FA_val, r])
        if pdesc == 'Non-affective,psychosis':
            wtnonaff_FA.append([FA_val, r])
    if pdesc == 'In,good,health':
        n_control += 1
    if pdesc == 'Non-affective,psychosis':
        n_nonaff +=1

In [17]:
wtc_regionwise_struct = []
wtn_rw_struct = []

all_wtFAs_c = []
mean_wtFAs_c = []
stdv_wtFA_c = []

all_wtFAs_n = []
mean_wtFAs_n = []
stdv_wtFA_n = []

curr_region = 0
for r in wtFA_filenames:
    wtregional_c = [tuple for tuple in wtcontrol_FA if any(r == i for i in tuple)] #Sort data by region
    wtreg_n = [tuple for tuple in wtnonaff_FA if any(r == i for i in tuple)]
    
    wtc_regionwise_struct.append(wtregional_c) #Create list of tuples for each group
    wtn_rw_struct.append(wtreg_n)

    fullc = [list(zip(*wtc_regionwise_struct[curr_region]))[0],r]
    meanc = [np.mean(list(zip(*wtc_regionwise_struct[curr_region]))[0]),r]
    stdvc = [np.std(list(zip(*wtc_regionwise_struct[curr_region]))[0]),r]
    fulln = [list(zip(*wtn_rw_struct[curr_region]))[0],r]
    meann = [np.mean(list(zip(*wtn_rw_struct[curr_region]))[0]),r]
    stdvn = [np.std(list(zip(*wtn_rw_struct[curr_region]))[0]),r]

    all_wtFAs_c.append(fullc) #Store all, mean, and standard deviation values in list including region labels
    mean_wtFAs_c.append(meanc)
    stdv_wtFA_c.append(stdvc)

    all_wtFAs_n.append(fulln)
    mean_wtFAs_n.append(meann)
    stdv_wtFA_n.append(stdvn)


    curr_region += 1 #Increase index counter

In [18]:
#Calculate mean and standard deviation for the FA values in each group and perform subsequent T-tests for differences in means with heterogenous variance
wtDifference_C_N = []
wtDifference_C_A = []
wtDifference_N_A = []
difference_c_n_fs = []
difference_c_a_fs = []
difference_n_a_fs = []

abs_wt_differences_cn = []
wt_cn_pvals = []
wt_cn_CI_L = []
wt_cn_CI_U = []

wtBC_dcn = []
wtBC_dca = []
wtBC_dna = []

wtall_pvals_cn = []
wtall_pvals_ca = []
wtall_pvals_na = []

wtdisc_cn = 0
wtdisc_ca = 0
wtdisc_na = 0

for r in enumerate(wtFA_filenames):
    #Difference in Control vs Non-affective Psychosis Fractional Anisotropy
    wttest_c_n = stats.ttest_ind(all_wtFAs_c[r[0]][0],all_wtFAs_n[r[0]][0],equal_var=False)
    wtabs_diff_cn = np.mean(all_wtFAs_c[r[0]][0]) - np.mean(all_wtFAs_n[r[0]][0])
    wtall_pvals_cn.append(wttest_c_n[1])
    if wttest_c_n[1] <= 0.05:
        store_wtc_n = [wtabs_diff_cn, wttest_c_n[0], wttest_c_n[1], r[1]]
        wtDifference_C_N.append(store_wtc_n)
        wtdisc_cn += 1
    if wttest_c_n[1] <= 0.05 / 50:
        storeBC_wtc_n = [wtabs_diff_cn, wttest_c_n[0], wttest_c_n[1], r[1]]
        wtBC_dcn.append(storeBC_wtc_n)
    #if wttest_c_n[1] <= FDR:
     #   storeFDR_wtc_n = [wtabs_diff_cn, wttest_c_n[0], wttest_c_n[1], r[1]]
     #   wtFDRC_dcn.append(storeFDR_wtc_n)
    
    abs_wt_differences_cn.append(wtabs_diff_cn)
    wt_cn_pvals.append(wttest_c_n[1])
    wt_cn_CI_L.append(wttest_c_n.confidence_interval(confidence_level=0.95)[0])
    wt_cn_CI_U.append(wttest_c_n.confidence_interval(confidence_level=0.95)[1])

In [19]:
fdr_corrected_weighted_pvals = mt.fdrcorrection(wtall_pvals_cn,alpha=0.05)

In [20]:
omuir_differences_fdr_corrected_table = pd.DataFrame(columns=['ID','\u0394','FDR P-val', 'CI-LB', 'CI-UB', 'P-val','Tal. Labels'])


In [40]:
shifted_wm_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [42]:
omuir_differences_fdr_corrected_table['ID'] = shifted_wm_list # Label IDs
omuir_differences_fdr_corrected_table['\u0394'] = abs_wt_differences_cn #Second column is absolute difference in FA
omuir_differences_fdr_corrected_table['FDR P-val'] = fdr_corrected_weighted_pvals[1] #Third column is corrected pval
omuir_differences_fdr_corrected_table['CI-LB'] = wt_cn_CI_L #Fourth column is one sided confidence interval
omuir_differences_fdr_corrected_table['CI-UB'] = wt_cn_CI_U
omuir_differences_fdr_corrected_table['P-val'] = wt_cn_pvals
omuir_differences_fdr_corrected_table['Tal. Labels'] = all_talairach_top_flags

In [43]:
omuir_differences_fdr_corrected_table.style \
.format(precision=6, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.applymap(bold_significant_ps, subset=['FDR P-val', 'P-val']) \
.hide()

ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
1,0.487189,0.128650,-0.008615,0.982993,0.054033,"(133.0, 173.0, 51.0)"
2,0.708969,0.136643,-0.030807,1.448744,0.060123,"(485.0, 88.0, 51.0)"
3,0.235981,0.413878,-0.211617,0.683579,0.297992,"(347.0, 51.0, 485.0)"
4,1.349789,0.102297,0.098148,2.601430,0.034781,"(879.0, 878.0, 784.0)"
5,0.919312,0.144390,-0.063345,1.901969,0.066419,"(195.0, 193.0, 155.0)"
6,0.600513,0.071217,0.088611,1.112414,0.021941,"(51.0, 157.0, 252.0)"
7,-0.059139,0.867530,-0.482208,0.363931,0.782257,"(4.0, 687.0, 194.0)"
8,0.371937,0.719713,-0.892852,1.636726,0.561376,"(796.0, 879.0, 878.0)"
9,0.111311,0.867530,-0.666988,0.889610,0.777318,"(193.0, 194.0, 195.0)"
10,1.444916,0.035906,0.447148,2.442683,0.004912,"(24.0, 51.0, 39.0)"


In [44]:
stripped_omuir_table = omuir_differences_fdr_corrected_table.drop(omuir_differences_fdr_corrected_table[omuir_differences_fdr_corrected_table['FDR P-val'] > 0.05].index)

In [55]:
stripped_omuir_table.style \
.format(precision=4, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.hide()

ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
10,1.4449,0.0359,0.4471,2.4427,0.0049,"(24.0, 51.0, 39.0)"
17,0.6983,0.0359,0.2254,1.1713,0.0043,"(134.0, 174.0, 664.0)"
22,0.6134,0.0359,0.1888,1.0381,0.0050,"(194.0, 348.0, 156.0)"
29,1.3953,0.0434,0.3748,2.4158,0.0078,"(89.0, 486.0, 253.0)"
37,1.3549,0.0457,0.3299,2.3800,0.0100,"(195.0, 193.0, 51.0)"
38,0.6813,0.0432,0.1907,1.1720,0.0069,"(798.0, 879.0, 796.0)"
42,0.9077,0.0307,0.3285,1.4870,0.0025,"(51.0, 308.0, 485.0)"
43,0.6037,0.0443,0.1546,1.0527,0.0089,"(189.0, 347.0, 193.0)"
46,1.6278,0.0281,0.6423,2.6132,0.0014,"(922.0, 107.0, 374.0)"
49,2.0302,0.0114,0.9767,3.0837,0.0002,"(39.0, 24.0, 51.0)"


In [25]:
#Grey matter label names
GM_name_dict = {3: 'L Sup. Postcentral + L_G Inf. Pariet (Supramarginal)' ,9:'L Inf Par (Angular Gyrus) & L Mid Temp Gyrus',\
 12: 'Bilateral Precuneus & Sup parietal gyri (supremarginal)',\
 14:'L_Insula, L_G Pariet Inf. Supramarginal', 15:'L_G Orbitofrontal - L Sup Frot.', 16:'R Entorhinal & Parahippocampal Gyri',\
 21:'R Superior Frontal, R Sup. Orbitofrontal, & R rostral middle frontal Gyri',22:'Bilat. Med. OF Gyri',23: 'R_G_OF - R Sup. Fron.',\
 28:'R_G Occipital-Temporal med Lingual & R_G Cuneus',36:'L_G Fron. Mid., Rost. Fron. Mid, Inf Opercular, G Temp. Inf., Caudal Mid. Fron.',\
 37:'Biltaeral Superior Parietal',41:'L lat. Occ. L Mid. Occ. L Pole Occ.',42:'L_G Insula',44:'R Insula + Sup Temp + R_G_ Pariet Inf Supramarginal',\
 45:'R Superior Frontal, R Cingulate Cortex',48:'L_G temp sup lat. + temp. mid. + par. inf. supramarginal',\
 49:'L_G temp. sup lateral + L_G occ. mid. L lat Fis post.'}

In [26]:
#stripped_omuir_table['GM Region'] = stripped_omuir_table['Label ID'].map(GM_name_dict)
#stripped_nwo_table['GM Region'] = stripped_nwo_table['Label ID'].map(GM_name_dict)

In [56]:
stripped_omuir_table.style \
.format(precision=4, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.hide()

ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
10,1.4449,0.0359,0.4471,2.4427,0.0049,"(24.0, 51.0, 39.0)"
17,0.6983,0.0359,0.2254,1.1713,0.0043,"(134.0, 174.0, 664.0)"
22,0.6134,0.0359,0.1888,1.0381,0.0050,"(194.0, 348.0, 156.0)"
29,1.3953,0.0434,0.3748,2.4158,0.0078,"(89.0, 486.0, 253.0)"
37,1.3549,0.0457,0.3299,2.3800,0.0100,"(195.0, 193.0, 51.0)"
38,0.6813,0.0432,0.1907,1.1720,0.0069,"(798.0, 879.0, 796.0)"
42,0.9077,0.0307,0.3285,1.4870,0.0025,"(51.0, 308.0, 485.0)"
43,0.6037,0.0443,0.1546,1.0527,0.0089,"(189.0, 347.0, 193.0)"
46,1.6278,0.0281,0.6423,2.6132,0.0014,"(922.0, 107.0, 374.0)"
49,2.0302,0.0114,0.9767,3.0837,0.0002,"(39.0, 24.0, 51.0)"


In [57]:
stripped_nwo_table.style \
.format(precision=4, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.hide()

ID,Δ,FDR P-VAL,CI-LB,CI-UB,P-VAL,TAL. LABELS
4,0.0076,0.0434,0.0016,0.0136,0.0139,"(879.0, 878.0, 784.0)"
13,0.0102,0.0375,0.0024,0.0180,0.0105,"(860.0, 864.0, 968.0)"
15,0.0119,0.0075,0.0051,0.0187,0.0007,"(347.0, 439.0, 39.0)"
16,0.0118,0.0308,0.0032,0.0203,0.0074,"(193.0, 347.0, 195.0)"
17,0.0101,0.0151,0.0038,0.0164,0.0021,"(134.0, 174.0, 664.0)"
22,0.0095,0.0151,0.0035,0.0154,0.0021,"(194.0, 348.0, 156.0)"
23,0.0106,0.0256,0.0032,0.0180,0.0056,"(230.0, 231.0, 107.0)"
24,0.0096,0.0414,0.0021,0.0171,0.0124,"(348.0, 231.0, 156.0)"
29,0.0115,0.0256,0.0034,0.0195,0.0055,"(89.0, 486.0, 253.0)"
37,0.0117,0.0178,0.0040,0.0194,0.0031,"(195.0, 193.0, 51.0)"


In [29]:
import dataframe_image as dfi   

In [30]:
pd.set_option('display.colheader_justify','center')

In [59]:
centered_del_nwo = stripped_nwo_table.style.format(precision=4,thousands=',',decimal='.').hide()
centered_del_nwo.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
centered_del_nwo.set_properties(**{'text-align': 'center'})

nwo_lat = centered_del_nwo.to_latex()
print(nwo_lat)
#pd.set_option('display.colheader_justify','center')
#dfi.export(centered_del_nwo,'/home/corey/thesis-EP/EP_scripts/analysis/plots/new_Omuir_stripped_nw_WMdiffs_nogm.png',table_conversion='matplotlib')

\begin{table}
\thcenter
\begin{tabular}{rrrrrrl}
ID & Δ & FDR P-val & CI-LB & CI-UB & P-val & Tal. Labels \\
\text-aligncenter 4 & \text-aligncenter 0.0076 & \text-aligncenter 0.0434 & \text-aligncenter 0.0016 & \text-aligncenter 0.0136 & \text-aligncenter 0.0139 & \text-aligncenter (879.0, 878.0, 784.0) \\
\text-aligncenter 13 & \text-aligncenter 0.0102 & \text-aligncenter 0.0375 & \text-aligncenter 0.0024 & \text-aligncenter 0.0180 & \text-aligncenter 0.0105 & \text-aligncenter (860.0, 864.0, 968.0) \\
\text-aligncenter 15 & \text-aligncenter 0.0119 & \text-aligncenter 0.0075 & \text-aligncenter 0.0051 & \text-aligncenter 0.0187 & \text-aligncenter 0.0007 & \text-aligncenter (347.0, 439.0, 39.0) \\
\text-aligncenter 16 & \text-aligncenter 0.0118 & \text-aligncenter 0.0308 & \text-aligncenter 0.0032 & \text-aligncenter 0.0203 & \text-aligncenter 0.0074 & \text-aligncenter (193.0, 347.0, 195.0) \\
\text-aligncenter 17 & \text-aligncenter 0.0101 & \text-aligncenter 0.0151 & \text-alignc

In [60]:

#dfi.export(stripped_nwo_table.style.format(precision=4,thousands=',',decimal='.').hide().set_properties(subset='\u0394', **{'text-align': 'center'}),'/home/corey/thesis-EP/EP_scripts/analysis/plots/Omuir_stripped_nw_WMdiffs_nogm.png',table_conversion='matplotlib')
dfi.export(centered_del_nwo,'/home/corey/thesis-EP/EP_scripts/analysis/plots/Omuir_stripped_nw_WMdiffs_nogm.png',table_conversion='matplotlib')

dfi.export(stripped_omuir_table.style.format(precision=4,thousands=',',decimal='.').hide(),'/home/corey/thesis-EP/EP_scripts/analysis/plots/Omuir_stripped_wt_WMdiffs_nogm.png',table_conversion='matplotlib')
dfi.export(omuir_differences_fdr_corrected_table.style.format(precision=4,thousands=',',decimal='.').hide(),'/home/corey/thesis-EP/EP_scripts/analysis/plots/Omuir_full_wt_WMdiffs_nogm.png',table_conversion='matplotlib')
dfi.export(nwomuir_differences_fdr_corrected_table.style.format(precision=4,thousands=',',decimal='.').hide(),'/home/corey/thesis-EP/EP_scripts/analysis/plots/Omuir_full_nw_WMdiffs_nogm.png',table_conversion='matplotlib')

findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: 

In [33]:
csvstorage_path = '/home/corey/thesis-EP/EP_scripts/analysis/plots/fa_tables/csvs/'
list_of_columns_omuir = ['Label ID','Difference in Means','FDR corrected P-val', '95% CI Lower Bound', '95% CI Upper Bound', 'Uncorrected P-val','Talairach Labels']

In [34]:
#stripped_nwo_table.to_csv(csvstorage_path + 'stripped_nwo_table.csv',columns=list_of_columns_omuir,index=False)

In [35]:
import scipy as sp

In [36]:
#Load in EC values to correlate against corresponding O'muircheartaigh and Jbabdi ICs
#pebANDbmr_file = '/media/corey/4TB-WDBlue/data-thesis/fMRI/bmrs/16July/pebsANDbmrs.mat'

#Need to import subject wise estimates of effective connectivity post BMR
#peb_data = sp.io.loadmat(pebANDbmr_file)

#bmr_av_arr = peb_data['peb_bmr_1']['Ep'][0].item()[0:441,0]
#bmr_diff_arr = peb_data['peb_bmr_1']['Ep'][0].item()[441:882,0]

#bmr_av_arr2 = peb_data['peb_bmr_2']['Ep'][0].item()[0:441,0]
#bmr_diff_arr2 = peb_data['peb_bmr_2']['Ep'][0].item()[441:882,0]

#reshaped_bmr_av_arr = np.reshape(bmr_av_arr,(21,21))
#reshaped_bmr_diff = np.reshape(bmr_diff_arr,(21,21))

#rs_bmr_2 = np.reshape(bmr_av_arr2,(21,21))
#rs_bmrd_2 = np.reshape(bmr_diff_arr2,(21,21))

#d_rs_bmr = reshaped_bmr_av_arr.toarray()
#d_rs_bmrd = reshaped_bmr_diff.toarray()

#Sum of two arrays (Schizophrenic group average EC)
#d_rs_bmr_scz = d_rs_bmrd + d_rs_bmr

#Same thing with data from second frequency band
#d_rs_bmr_2 = rs_bmr_2.toarray()
#d_rs_bmrd_2 = rs_bmrd_2.toarray()

#d_rs_bmr_scz_2 = d_rs_bmr_2 + d_rs_bmrd_2

In [37]:
import statsmodels as ssm

In [38]:
#Correlate FA values to Ep values from nodes most closely related to GM volumes specified in O'muircheartaigh and Jbabdi 2018

#PCC -> 45 (Whole right cingulate gyrus)
#pcc_x = [weighted_pos_inout_strengths_c1[1,1],weighted_pos_inout_strengths_c1]
#mPFC -> 22 (bilateral mPFC)

#LlPar -> 14

#RlPar -> 44

#LiTem -> 48

#RiTem -> 21?

#mdThal -> no analog

#LpCerb + RpCerb -> no analogs

#dmPFC -> 23?

#LaPFC -> 15

#RaPFC -> 21

#LsPar -> 3

#RsPar -> 44?

#dACC -> 45 (whole R CC)

#LaPFC_SN -> 15

#RaPFC_SN -> 21

#L_Ins -> 42

#R_Ins -> 44

#LlPar_SN -> 14

#RlPar_SN -> 44